In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from pin_utils import PinModel
from bam.descriptions import UR
import numpy as np

In [3]:
rd = UR.make_UR5e()
pin_model = PinRobotModel.from_robot_description(rd)

In [4]:
pin_model.inspect(print_model=True, print_joints=True, print_frames=True, print_inertias=True)

Model Name      : ur5e
Gravity         : [ 0.    0.   -9.81]

Weight (kg):
  universe: 4.00
  shoulder_pan_joint: 3.76
  shoulder_lift_joint: 8.06
  elbow_joint: 2.85
  wrist_1_joint: 1.37
  wrist_2_joint: 1.30
  wrist_3_joint: 0.36
  --
  Total: 21.70

Counts:
  Joints         : 7
  Bodies         : 7
  Frames         : 30
  Position (dof) : 6
  Velocity (dof) : 6
  Effort (dof)   : 6

Joints (n = 7):
  [0] universe, type: JointModelRX, parent id: 0
  [1] shoulder_pan_joint, type: JointModelRZ, parent id: 0
  [2] shoulder_lift_joint, type: JointModelRZ, parent id: 1
  [3] elbow_joint, type: JointModelRZ, parent id: 2
  [4] wrist_1_joint, type: JointModelRZ, parent id: 3
  [5] wrist_2_joint, type: JointModelRZ, parent id: 4
  [6] wrist_3_joint, type: JointModelRZ, parent id: 5
Joint Positions:  [0. 0. 0. 0. 0. 0.]
[i] Joint Name               :  Position xyz
[0] universe                 :  0.0000  0.0000  0.0000
[1] shoulder_pan_joint       :  0.0000  0.0000  0.1625
[2] shoulder_lift_j

In [5]:
pin_model.frames_exists(["ee_link", "tool0"])

try:
    pin_model.frames_exists(["not_a_real_frame"])
except Exception as e:
    print(e)



Frame not_a_real_frame not found in model


In [6]:
T_ee_link_to_tool0 = pin_model.transform_between_frames("ee_link", "tool0")
T_base_link_to_ee_link = pin_model.transform_between_frames("base_link", "ee_link")

T_base_link_to_tool0 = T_base_link_to_ee_link @ T_ee_link_to_tool0

try:
    T_error = T_ee_link_to_tool0 @ T_base_link_to_ee_link 
except Exception as e:
    print(e)



child_frame_id and frame_id mismatch:
    T_ee_link_to_tool0 @ T_base_link_to_ee_link



In [7]:
pin_model.get_name_to_joint_index()

{'shoulder_pan_joint': 0,
 'shoulder_lift_joint': 1,
 'elbow_joint': 2,
 'wrist_1_joint': 3,
 'wrist_2_joint': 4,
 'wrist_3_joint': 5}

In [8]:
pin_model.get_joint_names()

['shoulder_pan_joint',
 'shoulder_lift_joint',
 'elbow_joint',
 'wrist_1_joint',
 'wrist_2_joint',
 'wrist_3_joint']

In [54]:
import random

q = np.array([1, 2, 3, 4, 5, 6])
joint_names = pin_model.get_joint_names()


indices = list(range(len(joint_names)))
random.shuffle(indices)

shuffled_joint_names = [joint_names[i] for i in indices]
shuffled_q = q[indices]

print("Original joint_names:", joint_names)
print("Shuffled joint_names:", shuffled_joint_names)
print("Shuffled q         :", shuffled_q)

q_unshuffled = pin_model.order_q_by_name(shuffled_joint_names, shuffled_q)
print("Unshuffled q     :", q_unshuffled)



Original joint_names: ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Shuffled joint_names: ['wrist_2_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_3_joint', 'shoulder_pan_joint', 'wrist_1_joint']
Shuffled q         : [5 2 3 6 1 4]
Unshuffled q     : [1. 2. 3. 4. 5. 6.]


In [58]:
lower, upper = pin_model.get_q_limits()
neutral = pin_model.get_q_neutral()
random_q = pin_model.get_q_random()

print(lower)
print(upper)
print(neutral)
print(random_q)



[-3.14059265 -3.14059265 -3.14059265 -4.71138898 -4.71138898 -3.14059265]
[3.14059265 3.14059265 3.14059265 1.56979633 1.56979633 3.14059265]
[0. 0. 0. 0. 0. 0.]
[-2.27861569  1.91059056 -2.15646226 -2.19298294 -3.89615113 -2.45714441]
